In [2]:
# page12

def AND_gate(x1, x2):
  w1 = 0.5
  w2 = 0.5
  b = -0.7
  result = x1*w1 + x2*w2 + b
  if result <= 0:
    return 0
  else:
    return 1



In [3]:
AND_gate(0,0), AND_gate(0,1), AND_gate(1,0), AND_gate(1,1)

(0, 0, 0, 1)

In [4]:
# page14

def NAND_gate(x1, x2):
  w1 = -0.5
  w2 = -0.5
  b = 0.7
  result = x1*w1 + x2*w2 + b
  if result <= 0:
    return 0
  else:
    return 1

In [5]:
NAND_gate(0,0), NAND_gate(0,1), NAND_gate(1,0), NAND_gate(1,1)

(1, 1, 1, 0)

In [6]:
# page17

def OR_gate(x1, x2):
  w1 = 0.6
  w2 = 0.6
  b = -0.5
  result = x1*w1 + x2*w2 + b
  if result <= 0:
    return 0
  else:
    return 1

In [7]:
OR_gate(0,0), OR_gate(0,1), OR_gate(1,0), OR_gate(1,1)

(0, 1, 1, 1)

In [4]:
# page89 - 09-04 네거티브 샘플링을 이용한 Word2Vec 구현(Skip-Gram with Negative Sampling, SGNS)

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras.preprocessing.text import Tokenizer

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('총 샘플 수 :',len(documents))


총 샘플 수 : 11314


In [5]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ", regex=True)
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

news_df.isnull().values.any()


False

In [6]:
news_df.replace("", float("NaN"), inplace=True)
news_df.isnull().values.any()

True

In [7]:
news_df.dropna(inplace=True)
print('총 샘플 수 :',len(news_df))

총 샘플 수 : 10995


In [8]:
import nltk
nltk.download('stopwords')

# 불용어를 제거
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
tokenized_doc = tokenized_doc.to_list()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# 단어가 1개 이하인 샘플의 인덱스를 찾아서 저장하고, 해당 샘플들은 제거.

# tokenized_doc이 numpy 배열 형태라면 다시 리스트로 변환하여 처리하는 것이 안전합니다.
# 만약 tokenized_doc이 이미 리스트의 리스트 형태라면 이 변환 과정은 필요 없습니다.
if isinstance(tokenized_doc, np.ndarray):
  # numpy array가 객체 타입 배열일 경우 리스트로 다시 변환
  # 이 과정이 없다면 numpy 배열 내부의 list들을 처리할 때 문제가 될 수 있습니다.
  tokenized_doc = tokenized_doc.tolist()

# 단어 수가 1개 초과인 문장들만 필터링하여 새로운 리스트 생성
filtered_tokenized_doc = [sentence for sentence in tokenized_doc if len(sentence) > 1]

# 필터링된 리스트로 업데이트
tokenized_doc = filtered_tokenized_doc

print('총 샘플 수 :',len(tokenized_doc))


총 샘플 수 : 10940


In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_doc)

word2idx = tokenizer.word_index
idx2word = {value : key for key, value in word2idx.items()}
encoded = tokenizer.texts_to_sequences(tokenized_doc)


In [11]:
print(encoded[:2])

[[9, 59, 603, 207, 3278, 1495, 474, 702, 9470, 13686, 5533, 15227, 702, 442, 702, 70, 1148, 1095, 1036, 20294, 984, 705, 4294, 702, 217, 207, 1979, 15228, 13686, 4865, 4520, 87, 1530, 6, 52, 149, 581, 661, 4406, 4988, 4866, 1920, 755, 10668, 1102, 7837, 442, 957, 10669, 634, 51, 228, 2669, 4989, 178, 66, 222, 4521, 6066, 68, 4295], [1026, 532, 2, 60, 98, 582, 107, 800, 23, 79, 4522, 333, 7838, 864, 421, 3825, 458, 6488, 458, 2700, 4730, 333, 23, 9, 4731, 7262, 186, 310, 146, 170, 642, 1260, 107, 33568, 13, 985, 33569, 33570, 9471, 11491]]


In [12]:
vocab_size = len(word2idx) + 1
print('단어 집합의 크기 :', vocab_size)


단어 집합의 크기 : 64277


In [13]:
from tensorflow.keras.preprocessing.sequence import skipgrams
# 네거티브 샘플링
skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded[:10]]

# 첫번째 샘플인 skip_grams[0] 내 skipgrams로 형성된 데이터셋 확인
pairs, labels = skip_grams[0][0], skip_grams[0][1]
for i in range(5):
  print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
      idx2word[pairs[i][0]], pairs[i][0],
      idx2word[pairs[i][1]], pairs[i][1],
      labels[i]))


(holocaust (2669), ameo (55499)) -> 0
(rediculous (15227), reputation (5533)) -> 1
(soldiers (957), acid (3046)) -> 0
(lived (1148), inself (41961)) -> 0
(look (66), injustices (33257)) -> 0


In [14]:
print('전체 샘플 수 :',len(skip_grams))


전체 샘플 수 : 10


In [15]:
# 첫번째 뉴스그룹 샘플에 대해서 생긴 pairs와 labels의 개수
print(len(pairs))
print(len(labels))


2220
2220


In [16]:
skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded]


In [17]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input
from tensorflow.keras.layers import Dot
from tensorflow.keras.utils import plot_model
from IPython.display import SVG

embedding_dim = 100

# 중심 단어를 위한 임베딩 테이블
w_inputs = Input(shape=(1, ), dtype='int32')
word_embedding = Embedding(vocab_size, embedding_dim)(w_inputs)

# 주변 단어를 위한 임베딩 테이블
c_inputs = Input(shape=(1, ), dtype='int32')
context_embedding  = Embedding(vocab_size, embedding_dim)(c_inputs)

dot_product = Dot(axes=2)([word_embedding, context_embedding])
dot_product = Reshape((1,), input_shape=(1, 1))(dot_product)
output = Activation('sigmoid')(dot_product)

model = Model(inputs=[w_inputs, c_inputs], outputs=output)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam')
plot_model(model, to_file='model3.png', show_shapes=True, show_layer_names=True, rankdir='TB')

for epoch in range(1, 6):
  loss = 0
  for _, elem in enumerate(skip_grams):
    first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
    second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
    labels = np.array(elem[1], dtype='int32')
    X = [first_elem, second_elem]
    Y = labels
    loss += model.train_on_batch(X,Y)
  print('Epoch :',epoch, 'Loss :',loss)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 1, 100)    │  6,427,700 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 100)    │  6,427,700 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 1, 1)      │          0 │ embedding[0][0],  │
│                     │                   │            │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1)         │          0 │ dot[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 1)         │          0 │ reshape[0][0]     │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,855,400 (49.04 MB)

 Trainable params: 12,855,400 (49.04 MB)

 Non-trainable params: 0 (0.00 B)

Epoch : 1 Loss : 5387.5947033166885
Epoch : 2 Loss : 4608.700495183468
Epoch : 3 Loss : 4461.091066598892
Epoch : 4 Loss : 4380.402437537909
Epoch : 5 Loss : 4306.418601214886


In [20]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 21.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 req

In [ ]:
from gensim.models import Word2Vec

# 모델 학습 (예시)
# sentences = [['this', 'is', 'a', 'sentence'], ['another', 'sentence']]
# model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# 텍스트 형식으로 저장
# model.wv.save_word2vec_format('my_model.txt', binary=False)

# 바이너리 형식으로 저장
# model.wv.save_word2vec_format('my_model.bin', binary=True)

# 모델 자체를 Gensim 고유의 형식으로 저장 (학습 상태 및 내부 파라미터 포함)
# model.save('my_model.model')

# 저장된 텍스트 형식 파일 로드
# from gensim.models import KeyedVectors
# loaded_model_txt = KeyedVectors.load_word2vec_format('my_model.txt', binary=False)

# 저장된 바이너리 형식 파일 로드
# loaded_model_bin = KeyedVectors.load_word2vec_format('my_model.bin', binary=True)

# 저장된 Gensim 모델 자체 로드
# loaded_gensim_model = Word2Vec.load('my_model.model')

In [18]:
import gensim

f = open('vectors.txt' ,'w')
f.write('{} {}\n'.format(vocab_size-1, embedding_dim))
vectors = model.get_weights()[0]
for word, i in tokenizer.word_index.items():
  f.write('{} {}\n'.format(word, ' '.join(map(str, list(vectors[i, :])))))
f.close()

In [19]:
import gensim

# 모델 로드
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)


In [20]:
w2v.most_similar(positive=['soldiers'])


[('wounded', 0.7293126583099365),
 ('retalliates', 0.6837329268455505),
 ('shelling', 0.6696972250938416),
 ('villages', 0.6647743582725525),
 ('perpetrated', 0.6539343595504761),
 ('stationed', 0.652960479259491),
 ('israeli', 0.6523014903068542),
 ('parliament', 0.6519659757614136),
 ('districts', 0.6493146419525146),
 ('palestinian', 0.6405693292617798)]

In [21]:
w2v.most_similar(positive=['doctor'])


[('diagnosis', 0.6133856177330017),
 ('pain', 0.5959944128990173),
 ('codeine', 0.583656370639801),
 ('felbamate', 0.5815492868423462),
 ('cancer', 0.5752045512199402),
 ('intracranial', 0.568712055683136),
 ('quack', 0.557327389717102),
 ('patient', 0.5536763668060303),
 ('vehemently', 0.5534290075302124),
 ('syndromes', 0.5513232946395874)]

In [22]:
w2v.most_similar(positive=['police'])


[('filed', 0.5630268454551697),
 ('prisons', 0.546312153339386),
 ('understandingis', 0.528298556804657),
 ('officers', 0.5255241990089417),
 ('concealed', 0.5214292407035828),
 ('prohibit', 0.506625235080719),
 ('gurantee', 0.4998522996902466),
 ('uphold', 0.4923269748687744),
 ('illegally', 0.4887961447238922),
 ('negligence', 0.4873741567134857)]

In [23]:
w2v.most_similar(positive=['knife'])


[('wielder', 0.6540108323097229),
 ('deterred', 0.639933705329895),
 ('caplan', 0.6367617845535278),
 ('incompetance', 0.6297662854194641),
 ('cheek', 0.6247057318687439),
 ('pedieos', 0.6246527433395386),
 ('merrimanov', 0.623978853225708),
 ('reconcilication', 0.6227036118507385),
 ('gunshot', 0.6215704083442688),
 ('tyoe', 0.6210713386535645)]

In [24]:
w2v.most_similar(positive=['engine'])


[('gasket', 0.6453549861907959),
 ('carrera', 0.6030805110931396),
 ('gasoline', 0.584503710269928),
 ('dirt', 0.5809630751609802),
 ('carb', 0.5769891738891602),
 ('carburetor', 0.5721915364265442),
 ('pumps', 0.5705041289329529),
 ('aluminum', 0.5703490376472473),
 ('pickup', 0.5644032955169678),
 ('larger', 0.5568011403083801)]